In [1]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2

In [2]:
import gfw
import pandas as pd
from pathlib import Path
import geopandas as gpd
import time
from openmeteo import get_meteo

In [3]:
PATH_DATA = Path.cwd().parent.joinpath('data')

In [4]:
# Get list of vessel MMSI

vessels = pd.read_csv(PATH_DATA.joinpath('livestock_vessels.csv'), sep=';')
mmsi = list(set(vessels.mmsi))

In [ ]:
# Query GFW for vessel info

results = gfw.get_vessels(mmsi)
len(results)

In [ ]:
# Parse vessel data

vessels = []
owners = []

for result in results:
    entries = result.get('entries')
    for entry in entries:
        if entry.get('registryInfoTotalRecords') != 0:
            ships = entry.get('registryInfo')
        else:
            ships = entry.get('selfReportedInfo')
        vessels.append(ships[0])
        coms = entry.get('registryOwners')
        if len(coms) > 0:
            for com in coms:
                owners.append(com)
        
vessels = pd.DataFrame(vessels)
owners = pd.DataFrame(owners)

# Write to file
vessels.to_csv(PATH_DATA.joinpath('vessels.csv'), index=False)
owners.to_csv(PATH_DATA.joinpath('owners.csv'), index=False)

In [ ]:
# Get ids for events

ids = list(set(vessels.id))
len(ids)

## Get port visits

In [ ]:
port_visits = []

for id in ids:
    visits = gfw.get_events(id, event_type='port_visits')
    for entry in visits.get('entries'):
        port_visits.append(entry)

In [ ]:
# parse data
records = []
for entry in port_visits:
    
    record_dict = {'start': entry.get('start'),
                'end': entry.get('end'),
                'id': entry.get('id'),
                'lat': entry.get('position').get('lat'),
                'lon': entry.get('position').get('lon'),
                'vessel_id': entry.get('vessel').get('id'),
                'vessel_name': entry.get('vessel').get('name'),
                'mmsi': entry.get('vessel').get('ssvid'),
                'duration': entry.get('port_visit').get('durationHrs')
                }
    records.append(record_dict)

# Create geodataframe
vis = pd.DataFrame(records)
vis = gpd.GeoDataFrame(vis, geometry=gpd.points_from_xy(x=vis.lon, y=vis.lat), crs=4326)

# Write to file
vis.to_csv(PATH_DATA.joinpath('port_visits.csv'), index=False)

## Get loitering events

In [ ]:
loitering = []

for id in ids:
    try:
        visits = gfw.get_events(id, event_type='loitering')
    except ConnectionError:
        print(f'connection error at {id}')
        time.sleep(10)
    for entry in visits.get('entries'):
        loitering.append(entry)

In [ ]:
# Parse data

loiterings = []

for entry in loitering:
    record_dict = {'start': entry.get('start'),
                    'end': entry.get('end'),
                    'id': entry.get('id'),
                    'lat': entry.get('position').get('lat'),
                    'lon': entry.get('position').get('lon'),
                    'vessel_id': entry.get('vessel').get('id'),
                    'vessel_name': entry.get('vessel').get('name'),
                    'mmsi': entry.get('vessel').get('ssvid'),
                    'loitering_hours': entry.get('loitering').get('totalTimeHours')
                    }
    loiterings.append(record_dict)

df_loitering = pd.DataFrame(loiterings)

# Write to file
df_loitering.to_csv(PATH_DATA.joinpath('loitering.csv'), index=False)

## Get AIS gaps

In [ ]:

gaps = []

for i, id in enumerate(ids):
    try:
        result = gfw.get_events(id, 'ais')
    except ConnectionError:
        print(f'Connection Error with {id}')
        continue
    for entry in gaps:
        gaps.append(entry.get('entries'))
    